In [3]:
import pandas as pd
import json

json_file_path = "../results/medical_professions.json"

# Load JSON data from file
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

len(data)

21